In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix, mean_absolute_error, classification_report
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier,BaggingRegressor,BaggingClassifier, \
AdaBoostRegressor,AdaBoostClassifier,RandomForestRegressor,RandomForestClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingClassifier
from pyearth import Earth
import itertools as it
import time as time
import xgboost as xgb
import re 
from collections import Counter

import pandas as pd
import numpy as np
from sklearn import linear_model, metrics
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [5]:
X_train = pd.read_csv('X_train_stratified.csv')
y_train = pd.read_csv('y_train_stratified.csv')
X_test = pd.read_csv('X_test_stratified.csv')
y_test = pd.read_csv('y_test_stratified.csv')

In [6]:
# all models
dt_model = DecisionTreeClassifier(random_state=1, max_depth=17, max_features = 4, max_leaf_nodes=613).fit(X_train,y_train)
bg_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=1), n_estimators=300, random_state=1,max_features=7, 
                             max_samples=2500, bootstrap=False,bootstrap_features=False).fit(X_train, y_train)
rf_model = RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100, 
                                  max_features = 2, random_state=1).fit(X_train, y_train)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_xgb = le.fit_transform(y_train)

xgb_model = xgb.XGBClassifier(objective = 'multi:softmax',random_state = 1, gamma = 0.06, learning_rate = 0.3, max_depth = 9,
                              n_estimators = 150, reg_lambda = 1.0, subsample = 0.8, num_class = 7).fit(X_train, y_train_xgb)



/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/m0/5bk_k6ld73d91p0p4lfvfrvh0000gn/T/ipykernel_10997/1441565655.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model = RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100,
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/

In [17]:
#Voting ensemble: Averaging the predictions of all models
ensemble_model=VotingClassifier(estimators = [('dt',dt_model),('bg',bg_model),('rf',rf_model),('xgb',xgb_model)])
ensemble_model.fit(X_train,y_train)
ensemble_model.score(X_test, y_test)

/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6673469387755102

In [18]:
voting_pred = ensemble_model.predict(X_test)
print(classification_report(y_test, voting_pred))
print(confusion_matrix(y_test, voting_pred))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.69      0.27      0.39        33
           5       0.69      0.68      0.68       291
           6       0.63      0.78      0.70       440
           7       0.71      0.48      0.57       176
           8       1.00      0.54      0.70        35
           9       0.00      0.00      0.00         1

    accuracy                           0.67       980
   macro avg       0.53      0.39      0.44       980
weighted avg       0.68      0.67      0.66       980

[[  0   0   1   3   0   0   0]
 [  0   9  16   8   0   0   0]
 [  0   2 198  91   0   0   0]
 [  0   1  69 344  26   0   0]
 [  0   1   3  88  84   0   0]
 [  0   0   1   8   7  19   0]
 [  0   0   0   0   1   0   0]]


/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# train data predictions

In [63]:
# Train data:
dt_pred = dt_model.predict(X_train)
bg_pred = bg_model.predict(X_train)
rf_pred = rf_model.predict(X_train)
xgb_pred = xgb_model.predict(X_train)

In [64]:
# convert the xgb_predictions back to original classes
xgb_pred[xgb_pred == 0] = 3
xgb_pred[xgb_pred == 1] = 4
xgb_pred[xgb_pred == 2] = 5
xgb_pred[xgb_pred == 3] = 6
xgb_pred[xgb_pred == 4] = 7
xgb_pred[xgb_pred == 5] = 8
xgb_pred[xgb_pred == 6] = 9

In [65]:
all_preds = np.hstack((dt_pred.reshape(-1,1),bg_pred.reshape(-1,1),rf_pred.reshape(-1,1), xgb_pred.reshape(-1,1)))
all_preds

array([[7, 7, 7, 7],
       [7, 7, 7, 7],
       [5, 5, 5, 8],
       ...,
       [5, 5, 5, 8],
       [7, 7, 7, 7],
       [7, 6, 6, 9]])

In [66]:
majority_elements_train = []
for i in range(3918):
    c = Counter(list(all_preds[i]))
    c.most_common()
    value, count = c.most_common()[0]
    majority_elements_train.append(value)

In [69]:
print(classification_report(y_train, majority_elements_train))
print(confusion_matrix(y_train, majority_elements_train))

              precision    recall  f1-score   support

           3       1.00      1.00      1.00        16
           4       1.00      0.98      0.99       130
           5       1.00      1.00      1.00      1166
           6       1.00      1.00      1.00      1758
           7       1.00      1.00      1.00       704
           8       1.00      1.00      1.00       140
           9       1.00      1.00      1.00         4

    accuracy                           1.00      3918
   macro avg       1.00      1.00      1.00      3918
weighted avg       1.00      1.00      1.00      3918

[[  16    0    0    0    0    0    0]
 [   0  127    0    0    3    0    0]
 [   0    0 1166    0    0    0    0]
 [   0    0    0 1758    0    0    0]
 [   0    0    0    0  704    0    0]
 [   0    0    0    0    0  140    0]
 [   0    0    0    0    0    0    4]]


# test data predictions

In [81]:
# Test data:
dt_pred = dt_model.predict(X_test)
bg_pred = bg_model.predict(X_test)
rf_pred = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

In [82]:
# convert the xgb_predictions back to original classes
xgb_pred[xgb_pred == 0] = 3
xgb_pred[xgb_pred == 1] = 4
xgb_pred[xgb_pred == 2] = 5
xgb_pred[xgb_pred == 3] = 6
xgb_pred[xgb_pred == 4] = 7
xgb_pred[xgb_pred == 5] = 8
xgb_pred[xgb_pred == 6] = 9

In [83]:
print(classification_report(y_test, xgb_pred))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00        33
           5       0.00      0.00      0.00       291
           6       0.00      0.00      0.00       440
           7       0.60      0.53      0.56       176
           8       0.07      0.60      0.12        35
           9       0.00      0.00      0.00         1

    accuracy                           0.12       980
   macro avg       0.10      0.16      0.10       980
weighted avg       0.11      0.12      0.11       980



/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
all_preds = np.hstack((dt_pred.reshape(-1,1),bg_pred.reshape(-1,1),rf_pred.reshape(-1,1), xgb_pred.reshape(-1,1)))
all_preds.shape

(980, 4)

In [78]:
majority_elements_test = []
for i in range(980):
    c = Counter(list(all_preds[i]))
    c.most_common()
    value, count = c.most_common()[0]
    majority_elements_test.append(value)

In [79]:
# Test data:
print(classification_report(y_test, majority_elements_test))
print(confusion_matrix(y_test, majority_elements_test))

              precision    recall  f1-score   support

           3       0.00      0.00      0.00         4
           4       0.58      0.21      0.31        33
           5       0.72      0.64      0.68       291
           6       0.63      0.78      0.70       440
           7       0.64      0.52      0.58       176
           8       0.90      0.54      0.68        35
           9       0.00      0.00      0.00         1

    accuracy                           0.66       980
   macro avg       0.50      0.39      0.42       980
weighted avg       0.66      0.66      0.65       980

[[  0   0   1   3   0   0   0]
 [  0   7  15  10   1   0   0]
 [  1   3 186 100   1   0   0]
 [  0   1  55 344  40   0   0]
 [  0   0   3  79  92   2   0]
 [  0   1   0   7   8  19   0]
 [  0   0   0   0   1   0   0]]


/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# stacking ensemble

In [13]:
# all models
dt_model = DecisionTreeClassifier(random_state=1, max_depth=17, max_features = 4, max_leaf_nodes=613).fit(X_train,y_train)
bg_model = BaggingClassifier(base_estimator=DecisionTreeClassifier(random_state=1), n_estimators=300, random_state=1,max_features=7, 
                             max_samples=0.6380806533945891, bootstrap=False,bootstrap_features=False).fit(X_train, y_train)
rf_model = RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100, 
                                  max_features = 2, random_state=1).fit(X_train, y_train)

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_xgb = le.fit_transform(y_train)

xgb_model = xgb.XGBClassifier(objective = 'multi:softmax',random_state = 1, gamma = 0.06, learning_rate = 0.3, max_depth = 9,
                              n_estimators = 150, reg_lambda = 1.0, subsample = 0.8, num_class = 7).fit(X_train, y_train_xgb)



/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:719: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/var/folders/m0/5bk_k6ld73d91p0p4lfvfrvh0000gn/T/ipykernel_10997/3463990832.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model = RandomForestClassifier(criterion='entropy', max_depth=20, min_samples_leaf=1, min_samples_split=2, n_estimators=100,
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
#Using Logistic regression as the meta model (final_estimator)
ensemble_model = StackingClassifier(estimators=[('dt',dt_model),('bg',bg_model),('rf',rf_model),('xgb',xgb_model)],
                                   final_estimator=LogisticRegression(random_state=1,max_iter=10000),n_jobs=-1,
                                   cv = StratifiedKFold(n_splits=2,shuffle=True,random_state=1))
ensemble_model.fit(X_train,y_train)
ensemble_model.score(X_test, y_test)

/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/aartipappu/opt/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6602040816326531

In [11]:
X_train.shape

(3918, 11)

In [12]:
2500/3918

0.6380806533945891